In [408]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [615]:
N = 1000
X = np.random.randn(N * 2).reshape(N, 2) - 1
X[:N//2] = X[:N//2] + 2
y = np.array([0] * (N//2) + [1] * (N//2))
# px.histogram(x=X, color=y, barmode='overlay')
px.scatter(x=X[:, 0], y=X[:, 1], color=y)

In [616]:
X = np.hstack([np.ones(len(X)).reshape(N, 1), X])

In [617]:
def sigmoid(X, beta):
    exp = np.exp(np.dot(X, -beta))
    return 1 / (1 + exp)

In [618]:
def cost_function(X, y, beta):
    return np.dot(X.T, (y - sigmoid(X, beta)))

cost_function(X, y, np.array([0, 1, 2]))

array([   6.08821822, -935.06589109, -985.82587539])

In [619]:
def gradient_descent(X, y, n_iter=1000, l_rate=1e-5):
    N, p = X.shape
    # initial betas
    beta = np.random.randn(p)
    
    for i in range(n_iter):
        cost = cost_function(X, y, beta)
        beta += cost * l_rate
        
    return beta

In [620]:
def calculate_scores(y_true, y_pred):
    unique_classes = np.unique(y_true)
    conf_matrix = np.zeros([len(unique_classes), len(unique_classes)])
    conf_matrix[0, 0] = np.sum(pred[y_true==0] == 0)
    conf_matrix[1, 1] = np.sum(pred[y_true==1] == 1)
    conf_matrix[0, 1] = np.sum(pred[y_true==0] == 1)
    conf_matrix[1, 0] = np.sum(pred[y_true==1] == 0)
    
    acc = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    return conf_matrix, acc
#     for cls in unique_classes:
#         for cls_1 in unique_classes:
#             conf_matrix[cls]

In [621]:
beta = gradient_descent(X, y, n_iter=1000, l_rate=1e-2)
print(f'Estimated betas: {beta}')
pred_proba = sigmoid(X, beta)
pred = (pred_proba > 0.5).astype(int)
calculate_scores(y, pred)


Estimated betas: [-0.05757577 -2.08913408 -1.91613376]


(array([[460.,  40.],
        [ 39., 461.]]),
 0.921)

In [670]:
xx = np.linspace(-5, 5, 100)
yy = np.linspace(-5, 5, 100)
xx, yy = np.meshgrid(xx, yy)
xx, yy = xx.flatten(), yy.flatten()
mock_data = np.array([np.ones(len(xx)), xx, yy]).T
values = sigmoid(mock_data, beta)
go.Figure(
    [
        go.Contour(x=xx, y=yy, z=values),
        go.Scatter(x=X[y==0, 1], y=X[y==0, 2], mode='markers', marker_color='red'),
        go.Scatter(x=X[y==1, 1], y=X[y==1, 2], mode='markers', marker_color='blue')
    ],
    layout=dict(
        xaxis=dict(scaleanchor='y', scaleratio=1),
        width=600, height=600
    )
)

In [679]:
list(range(0, 1000, 50))

[0,
 50,
 100,
 150,
 200,
 250,
 300,
 350,
 400,
 450,
 500,
 550,
 600,
 650,
 700,
 750,
 800,
 850,
 900,
 950]

In [683]:
def SGD(X, y, n_iter=1000, l_rate=1e-3, n_chunk=50):
    N, p = X.shape
    # initial betas
    beta = np.random.randn(p)
    
    for _ in range(n_iter):
        ind = np.arange(N)
        np.random.shuffle(ind)
        for i in range(0, N, n_chunk):
            subset = ind[i:min(i + n_chunk, N) + 1]
            cost = cost_function(X, y, beta)
            update = cost * l_rate
            beta += update
        
    return beta
SGD(X, y)

array([-0.05757577, -2.08913408, -1.91613376])

In [676]:
def IRLS(X, y, stop_thresh=1e-3, max_iter=50):
    N, p = X.shape
    beta = np.zeros(p)
    for i in range(max_iter):
        p = sigmoid(X, beta)
        W = np.diag(p * (1 - p))
        tmp = X.T @ W
        beta_new = np.linalg.inv(tmp @ X) @ tmp @ (X @ beta + np.linalg.inv(W) @ (y - p))
        if np.linalg.norm(beta - beta_new) < stop_thresh:
            return beta
        beta = beta_new
    return beta
beta = IRLS(X, y)


In [677]:
xx = np.linspace(-5, 5, 100)
yy = np.linspace(-5, 5, 100)
xx, yy = np.meshgrid(xx, yy)
xx, yy = xx.flatten(), yy.flatten()
mock_data = np.array([np.ones(len(xx)), xx, yy]).T
values = sigmoid(mock_data, beta)
go.Figure(
    [
        go.Contour(x=xx, y=yy, z=values),
        go.Scatter(x=X[y==0, 1], y=X[y==0, 2], mode='markers', marker_color='red'),
        go.Scatter(x=X[y==1, 1], y=X[y==1, 2], mode='markers', marker_color='blue')
    ],
    layout=dict(
        xaxis=dict(scaleanchor='y', scaleratio=1),
        width=600, height=600
    )
)